In [1]:
import os
import json
import requests
import pprint
from warrant.aws_srp import AWSSRP

# Step 1: Get Token if you haven't previously

- _client_id_ = id of the Client App (In this caase the HomeControlPanel in Cognito)
- _user_pool_id_ = Watchdog User Pool id

This gives you _tokens_ which is a dict object with the Access token, Refresh Token, and Id Token

The Access Token can always be found in: 
```python
    tokens["AuthenticationResult"]["AccessToken"]
```
And this needs to to be send as a _Authorization_ Header

In [2]:
client_id="5bl2caob065vqodmm3sobp3k7d"
client_secret = None
user_pool_id = "eu-west-1_mQ0D78123"

aws = AWSSRP(username='test', password='Test123@', pool_id=user_pool_id,
             client_id=client_id, pool_region='eu-west-1')
tokens = aws.authenticate_user()

- *_get_location_url_* is the route in the api
- *_upload_key_* is the name of the artifact
- *_folder_* is the folder you would like to store it in

In [17]:
get_location_url = "https://la7nxehzwg.execute-api.af-south-1.amazonaws.com/alphav2/live/uploadclip/"
upload_key = "test_prefilled_post.txt"
folder = "video"

*Firstly*, you get the presigned link.
- This returns all the fields you require to upload to the S3 bucket


In [18]:
resp = requests.post(get_location_url, params={"upload_key": upload_key, "folder": folder}, headers={'Authorization': f'TOK:{tokens["AuthenticationResult"]["AccessToken"]}'})
print(f'{resp} : {resp.reason}')

<Response [200]> : OK


In [19]:
response_dict = json.loads(resp.text)

In [20]:
http_response = None
with open(upload_key, 'rb') as binary_object:
    files = {'file': (upload_key, binary_object)}
    http_response = requests.post(response_dict['url'], data=response_dict['fields'], files=files)

In [21]:
print(f'{http_response}: {http_response.reason}')

<Response [204]>: No Content
